In [21]:
# Jenny branch
import numpy as np

import datetime

import tensorflow as tf 

from sklearn import linear_model, ensemble, tree, svm, naive_bayes
from scipy.stats import uniform as sp_rand

import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Flatten, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.model_selection import RandomizedSearchCV

from collections import Counter



BORDER = "===================================================================================="

In [2]:
def load_data(filename, skiprows = 1):
    """
    Function loads data stored in the file filename and returns it as a numpy ndarray.
    
    Inputs:
        filename: given as a string.
        
    Outputs:
        Data contained in the file, returned as a numpy ndarray
    """
    return np.loadtxt(filename, skiprows=skiprows, delimiter=' ')

In [3]:
def WritePredictionOut(modelName, prediction):
    print(prediction)
    print("prediction shape is: {}".format(prediction.shape))

    output = "Id,Prediction\n"
    for i in range(prediction.shape[0]):
        output = output + ("{0},{1}\n".format(i + 1, prediction[i].astype(int)))

        
    now = datetime.datetime.now();
        
    filename = "{}_{}.{}.{}_{}.{}.{}_predictions.csv".format(modelName, now.year, now.month, now.day, now.hour, now.minute, now.second)
        
    file = open(filename,'w') 
    file.write(output)
    file.close()

In [4]:
def ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    if (verbose == 1):
        print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    # Layer set up.
    model = Sequential()
    
    model.add(Dense(1024, input_shape=(n_words, )))
    model.add(Activation('relu'))

    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    if (verbose == 1):
        model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=256, epochs=2, verbose=verbose, validation_split=0.1)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    if (verbose == 1):
        print('Training score:', score[0])
        print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=verbose)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [5]:
def ClassifyWithConvolutionNetwork(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using our neural network implementation
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    if (verbose == 1):
        print('\n{}\nNEURAL NETWORK\n{}\n'.format(BORDER, BORDER))

    # Converting Y values to one hot vector
    num_classes = 2
    Y_train = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # Input size
    n_train = X_train.shape[0]
    n_words = X_train.shape[1]
    n_test = Y_train.shape[0]
    
    
    # For 1D Convolution, change dimension of input 
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
        
    ## Create your own model here given the constraints in the problem
    model = Sequential()

    model.add(Conv1D(64, 3, padding="same", input_shape=(n_words, 1, )))
    
    model.add(Conv1D(32, 3, padding="same"))
    
    model.add(Conv1D(16, 3, padding="same"))

    model.add(Flatten())
    model.add(Dropout(0.2))
    model.add(Dense(120))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))

    ## Once you one-hot encode the data labels, the line below should be predicting probabilities of each of the 2 classes
    model.add(Dense(2))
    model.add(Activation('softmax'))

    ## Printing a summary of the layers and weights in your model
    if (verbose == 1):
        model.summary()

    ## In the line below we have specified the loss function as 'mse' (Mean Squared Error) because in the above code we did not one-hot encode the labels.
    ## In your implementation, since you are one-hot encoding the labels, you should use 'categorical_crossentropy' as your loss.
    ## You will likely have the best results with RMS prop or Adam as your optimizer.  In the line below we use Adadelta
    model.compile(loss='categorical_crossentropy',optimizer='RMSprop', metrics=['accuracy'])

    fit = model.fit(X_train, Y_train, batch_size=128, epochs=2, verbose=verbose, validation_split=0.1)
    
    ## Printing the accuracy of our model, according to the loss function specified in model.compile above
    score = model.evaluate(X_train, Y_train, verbose=0)
    
    if (verbose == 1):
        print('Training score:', score[0])
        print('Training accuracy:', score[1])

    prediction = model.predict(X_test, verbose=verbose)

    zeros = prediction[:, 0]
    ones  = prediction[:, 1]

    prediction = (zeros < ones).astype(int)

    return model, prediction, score[1]

In [6]:
def ClassifyWithLogisticRegression(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nLOGISTIC REGRESSION\n{}\n'.format(BORDER, BORDER))
    
    model = linear_model.LogisticRegression()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [7]:
def ClassifyWithLinearRegression(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nLINEAR REGRESSION\n{}\n'.format(BORDER, BORDER))
    
    model = linear_model.LinearRegression()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [8]:
def ClassifyWithAdaBoost(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """

    if (verbose == 1):
        print('\n{}\nADABOOST\n{}\n'.format(BORDER, BORDER))
    
    model = ensemble.AdaBoostClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [9]:
def ClassifyWithDecisionTree(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nDECISION TREE\n{}\n'.format(BORDER, BORDER))
        
    model = tree.DecisionTreeClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [10]:
def ClassifyWithRandomForest(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nRANDOM FOREST\n{}\n'.format(BORDER, BORDER))
        
    model = ensemble.RandomForestClassifier()
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [11]:
def ClassifyWithSVM(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nSVM\n{}\n'.format(BORDER, BORDER))
        
    model = svm.LinearSVC(verbose=verbose)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [12]:
def ClassifyWithSVC(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nSVC\n{}\n'.format(BORDER, BORDER))
        
    model = svm.SVC(kernel="rbf", C=0.03, verbose=verbose)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [13]:
def ClassifyWithBernoulliNB(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\nBernoulli Naive Bayes\n{}\n'.format(BORDER, BORDER))
        
    model = naive_bayes.BernoulliNB(alpha=.01)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [14]:
def ClassifyWithMultinomialNB(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        model: passes out the model we trained.
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
        accuracy: training accuracy of model.
    """
    
    if (verbose == 1):
        print('\n{}\n Multinomial Naive Bayes\n{}\n'.format(BORDER, BORDER))
        
    model = naive_bayes.MultinomialNB(alpha=.01)
    model.fit(X_train, Y_train)
    
    Y_pred = model.predict(X_train)
    
    correctlyClassified = (Y_pred == Y_train).astype(int);
    accuracy = np.sum(correctlyClassified) / correctlyClassified.shape[0]
    
    if (verbose == 1):
        print('Training accuracy: ', accuracy)
    
    prediction = model.predict(X_test)
    return model, prediction, accuracy

In [53]:
def ClassifyWithBlending(X_train, Y_train, X_test, verbose=0):
    """
    Function takes training and testing data, and fits using logistic regression
    
    Inputs:
        X_train: The training data
        Y_train: The training data labels
        X_test:  The testing data
        
    Outputs:
        prediction: a nparray with shape (X_test.shape[0], ) containing the predicted labels our model generated
    """
    
    if (verbose == 1):
        print('\n{}\nBlending\n{}\n'.format(BORDER, BORDER))
        
    _, pred_NN, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred_CN, _ = ClassifyWithConvolutionNetwork(X_train, Y_train, X_test, verbose=verbose)
    _, pred_LR, _ = ClassifyWithLogisticRegression(X_train, Y_train, X_test, verbose=verbose)
    _, pred_SVM, _ = ClassifyWithSVM(X_train, Y_train, X_test, verbose=verbose)
    _, pred_NN1, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=verbose)
        
    # Combine prediction into list
    preds = []
    for i in range(len(pred_NN)):
        preds.append((pred_NN[i], pred_CN[i], pred_LR[i], pred_SVM[i], pred_NN1[i]))
            
    
    # Final prediction
    Y_pred = np.array([])
    for i in range(len(preds)):
        lst = Counter(preds[i]).most_common()
        Y_pred = np.append(Y_pred, int(lst[0][0]))
    
    return Y_pred

In [15]:
def CrossValidation_NeuralNetwork(X_train, Y_train, X_test, nFold, verbose=0):
    
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
     # Converting Y values to one hot vector
    num_classes = 2
    Y = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model, prediction, accuracy = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_test)
        
        # Compute accuracy.
        val_acc = model.evaluate(X_train[vali], Y[vali], verbose=verbose)
            
        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc[1])

    print(BORDER)
    print("CROSS VALIDATION: ClassifyWithNeuralNetwork")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / 5.)
    print("average val accuracy", np.sum(total_val_acc) / 5.)
    
    return 0


In [16]:
def CrossValidation_ConvolutionNetwork(X_train, Y_train, X_test, nFold, verbose=0):
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    # Converting Y values to one hot vector
    num_classes = 2
    Y = keras.utils.to_categorical(Y_train, num_classes=num_classes)
    
    
    # For 1D Convolution, change dimension of input 
    X = np.expand_dims(X_train, axis=2)
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model, prediction, accuracy = ClassifyWithConvolutionNetwork(X_train[traini], Y_train[traini],X_test)
                
        # Compute accuracy
        val_acc = model.evaluate(X[vali], Y[vali], verbose=verbose)

        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc[1])

    print(BORDER)
    print("CROSS VALIDATION: ClassifyWithConvolutionNetwork")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / 5.)
    print("average val accuracy", np.sum(total_val_acc) / 5.)
    
    return 0

In [17]:
def CrossValidation_sklearn(modelFunction, X_train, Y_train, X_test, nFold, verbose=0):
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model, prediction, accuracy = modelFunction(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        
        # Compute accuracy.
        val_acc = model.score(X_train[vali], Y_train[vali])
            
        total_train_acc = np.append(total_train_acc, accuracy)
        total_val_acc = np.append(total_val_acc, val_acc)

    print(BORDER)
    print("CROSS VALIDATION: " + modelFunction.__name__)
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / float(nFold))
    print("average val accuracy", np.sum(total_val_acc) / float(nFold))
    
    return 0


In [54]:
def CrossValidation_Blending(X_train, Y_train, X_test, nFold, verbose=0):
    nModels = 4
    kf = KFold(n_splits = nFold)
    inds = [ind for ind in kf.split(X_train, Y_train)]
        
    total_train_acc = []
    total_val_acc = []
    
    # perform 5-fold validation
    for i in range(0,nFold):
        traini, vali = inds[i]
        model_NN, _, acc_NN = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        model_CN, _, acc_CN = ClassifyWithConvolutionNetwork(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        model_LR, _, acc_LR = ClassifyWithLogisticRegression(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        model_SVM, _, acc_SVM = ClassifyWithSVM(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        model_NN1, _, acc_NN1 = ClassifyWithNeuralNetwork(X_train[traini], Y_train[traini], X_test, verbose=verbose)
        
        
        # Compute prediction
        pred_NN = model_NN.predict(X_train[vali], verbose=verbose)
        zeros = pred_NN[:, 0]
        ones  = pred_NN[:, 1]
        pred_NN = (zeros < ones).astype(int)
        
        pred_CN = model_CN.predict(np.expand_dims(X_train[vali], axis=2), verbose=verbose)
        zeros = pred_CN[:, 0]
        ones  = pred_CN[:, 1]
        pred_CN = (zeros < ones).astype(int)
        
        # Compute prediction
        pred_NN1 = model_NN1.predict(X_train[vali], verbose=verbose)
        zeros = pred_NN1[:, 0]
        ones  = pred_NN1[:, 1]
        pred_NN1 = (zeros < ones).astype(int)
        
        pred_LR = model_LR.predict(X_train[vali])
        pred_SVM = model_SVM.predict(X_train[vali])
        
        # Combine prediction into list
        preds = []
        for i in range(len(pred_NN)):
            preds.append((pred_NN[i], pred_CN[i], pred_LR[i], pred_SVM[i], pred_NN1[i]))
            
    
        # Final prediction
        Y_pred = np.array([])
        for i in range(len(preds)):
            lst = Counter(preds[i]).most_common()
            Y_pred = np.append(Y_pred, int(lst[0][0]))
            
        total_train_acc = np.append(total_train_acc, acc_NN)
        total_train_acc = np.append(total_train_acc, acc_CN)
        total_train_acc = np.append(total_train_acc, acc_LR)
        total_train_acc = np.append(total_train_acc, acc_SVM)
        total_train_acc = np.append(total_train_acc, acc_NN1)
        
        correctlyClassified = (Y_pred == Y_train[vali]).astype(int);
        val_acc = np.sum(correctlyClassified) / correctlyClassified.shape[0]
        total_val_acc = np.append(total_val_acc, val_acc)

    print(BORDER)
    print("CROSS VALIDATION: Blending")
    print(BORDER)
    print("training accuracy", total_train_acc)
    print("val accuracy", total_val_acc)
    print("average training accuracy", np.sum(total_train_acc) / float(nModels * nFold))
    print("average val accuracy", np.sum(total_val_acc) / float(nFold))

In [18]:
# Loads training dataset
training = load_data('data/training_data.txt', 1)
X_train = training[:, 1:]
Y_train = training[:, 0]

In [19]:
# Loads testing dataset
# There is no label for testing set 
X_test = load_data('data/test_data.txt', 1)

In [ ]:
# Using Validation   
# CrossValidation_NeuralNetwork(X_train, Y_train, X_test, 5)
# CrossValidation_ConvolutionNetwork(X_train, Y_train, X_test, 5)

# CrossValidation_sklearn(ClassifyWithLinearRegression, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithLogisticRegression, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithAdaBoost, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithDecisionTree, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithRandomForest, X_train, Y_train, X_test, 5)

# CrossValidation_sklearn(ClassifyWithSVM, X_train, Y_train, X_test, 5, verbose=1)
# CrossValidation_sklearn(ClassifyWithSVC, X_train, Y_train, X_test, 5, verbose=1)

# CrossValidation_sklearn(ClassifyWithBernoulliNB, X_train, Y_train, X_test, 5)
# CrossValidation_sklearn(ClassifyWithMultinomialNB, X_train, Y_train, X_test, 5)

CrossValidation_Blending(X_train, Y_train, X_test, 5, verbose=1)


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_91 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
activation_91 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_92 (Dense)             (None, 2)                 2050      
_________________________________________________________________
activation_92 (Activation)   (None, 2)                 0         
Total params: 1,027,074
Trainable params: 1,027,074
Non-trainable params: 0
_________________________________________________________________
Train on 14400 samples, validate on 1600 samples
Epoch 1/2
14400/14400 [==============================] - 5s 379us/step - loss: 0.4296 - acc: 0.8103 - val_loss: 0.3583 - val_acc: 0.8556
Epoch 2/2
14400/14400 [==============================] - 3s 2

/Users/Jenny/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/svm/base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Training accuracy:  0.8766875

NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_95 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
activation_95 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_96 (Dense)             (None, 2)                 2050      
_________________________________________________________________
activation_96 (Activation)   (None, 2)                 0         
Total params: 1,027,074
Trainable params: 1,027,074
Non-trainable params: 0
_________________________________________________________________
Train on 14400 samples, validate on 1600 samples
Epoch 1/2
14400/14400 [==============================] - 5s 333us/step - loss: 0.4287 - acc: 0.8108 - val_loss: 0.3542 - val_acc: 0.8556
Epoch 2/2
14400/14400 [========

In [49]:
_, pred, _ = ClassifyWithNeuralNetwork(X_train, Y_train, X_test, verbose=1)
WritePredictionOut('NeuralNetwork', pred)


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_81 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
activation_81 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_82 (Dense)             (None, 2)                 2050      
_________________________________________________________________
activation_82 (Activation)   (None, 2)                 0         
Total params: 1,027,074
Trainable params: 1,027,074
Non-trainable params: 0
_________________________________________________________________
Train on 18000 samples, validate on 2000 samples
Epoch 1/2
18000/18000 [==============================] - 6s 317us/step - loss: 0.4109 - acc: 0.8204 - val_loss: 0.3526 - val_acc: 0.8560
Epoch 2/2
18000/18000 [==============================] - 4s 2

In [50]:
pred.shape

(10000,)

In [52]:
pred = ClassifyWithBlending(X_train, Y_train, X_test, verbose=1)
WritePredictionOut('Blending', pred)


Blending


NEURAL NETWORK

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_87 (Dense)             (None, 1024)              1025024   
_________________________________________________________________
activation_87 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_88 (Dense)             (None, 2)                 2050      
_________________________________________________________________
activation_88 (Activation)   (None, 2)                 0         
Total params: 1,027,074
Trainable params: 1,027,074
Non-trainable params: 0
_________________________________________________________________
Train on 18000 samples, validate on 2000 samples
Epoch 1/2
18000/18000 [==============================] - 7s 407us/step - loss: 0.4162 - acc: 0.8165 - val_loss: 0.3460 - val_acc: 0.8680
Epoch 2/2
18000/18000 [===========================

/Users/Jenny/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/svm/base.py:898: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Training accuracy:  0.87635
[1. 1. 0. ... 0. 1. 0.]
prediction shape is: (10000,)
